In [1]:
import tensorflow as tf

In [22]:
class NN:
    def __init__(
        self,
        layers,
        activations=None,
        weights=None,
        weight_initializer=None,
    ):
        self.layers = layers
        n_layers = len(layers) - 1

        if activations is None:
            self.activations = [self.sigmoid] * n_layers
        else:
            if len(activations) != n_layers:
                raise ValueError("activations must have length equal to len(layers) - 1")
            self.activations = activations

        self.weight_initializer = weight_initializer if weight_initializer else self.default_initializer
        self._weights = weights if weights is not None else self.init_weights()

        self.lr = None
        self.epochs = None

    def default_initializer(self, shape):
        return tf.Variable(tf.random.uniform(shape, minval=-1, maxval=1))

    def init_weights(self):
        weights = []
        for i in range(len(self.layers) - 1):
            W = self.weight_initializer((self.layers[i], self.layers[i+1]))
            b = self.weight_initializer((1, self.layers[i+1]))
            weights.append((W, b))
        return weights

    @staticmethod
    def sigmoid(x):
        return 1 / (1 + tf.exp(-x))

    def forward(self, x):
        out = x
        for idx, (W, b) in enumerate(self._weights):
            out = out @ W + b
            activation = self.activations[idx]
            if activation is not None:
                out = activation(out)
        return out

    def fit(self, x, y, epochs=20, lr=0.1, verbose=True):
        self.lr = lr
        self.epochs = epochs
        for i in range(1, self.epochs + 1):
            with tf.GradientTape() as tape:
                y_pred = self.forward(x)
                loss = tf.reduce_mean(tf.square(y - y_pred))
            grads = tape.gradient(
                loss,
                [param for layer in self._weights for param in layer]
            )
            for layer_idx, (W, b) in enumerate(self._weights):
                W.assign_sub(self.lr * grads[layer_idx * 2])
                b.assign_sub(self.lr * grads[layer_idx * 2 + 1])
            if verbose:
                print(f"{i:3d}. MSE={loss.numpy():.6f}")

    def predict(self, x):
        return self.forward(x)
    
    @property
    def weights(self):
        return [(W.numpy(), b.numpy()) for (W, b) in self._weights]

    @weights.setter
    def weights(self, weights):
        for i, (W_new, b_new) in enumerate(weights):
            W, b = self._weights[i]
            W.assign(W_new)
            b.assign(b_new)

    def print_weights(self):
        for i, (W, b) in enumerate(self._weights):
            print(f"Layer {i+1}:")
            print(f"  Weights:\n{W.numpy()}")
            print(f"  Biases:\n{b.numpy()}")

    def layer_weights(self, layer_idx):
        if layer_idx < 0 or layer_idx >= len(self._weights):
            raise IndexError("Invalid layer index")
        W, b = self._weights[layer_idx]
        return W.numpy(), b.numpy()

In [23]:
layers = [2, 2, 1]

W1 = tf.Variable([[0.15, 0.2], [0.25, 0.3]], dtype=tf.float32)
b1 = tf.Variable([[0.35, 0.45]], dtype=tf.float32)

W2 = tf.Variable([[0.4], [0.5]], dtype=tf.float32)
b2 = tf.Variable([[0.6]], dtype=tf.float32)

nn_weights = [
    (W1, b1),
    (W2, b2)
]

nn = NN(layers=layers, activations=[None, NN.sigmoid], weights=nn_weights)

In [24]:
x_train = tf.constant([[0.05, 0.1]], dtype=tf.float32)
y_train = tf.constant([[0.5]], dtype=tf.float32)

nn.fit(x_train, y_train, epochs=1, lr=0.01)

  1. MSE=0.053206


In [25]:
nn.print_weights() 

Layer 1:
  Weights:
[[0.14998184 0.19997731]
 [0.24996369 0.29995462]]
  Biases:
[[0.34963685 0.44954604]]
Layer 2:
  Weights:
[[0.39965275]
 [0.49955514]]
  Biases:
[[0.5990921]]
